# Search FORS2 in ESO catalogs 

http://eso-python.github.io/ESOPythonTutorials/ESOPythonDemoDay7_Astroquery_ESO_Simbad_Vizier.html


- access :

https://www.eso.org/UserPortal/


https://www.eso.org/sso/login?service=https%3A%2F%2Fwww.eso.org%2Fsso%2Foauth2.0%2FcallbackAuthorize%3Fclient_id%3Dclientid%26redirect_uri%3Dhttps%253A%252F%252Fwww.eso.org%252FUserPortal%252Fsignin%26response_type%3Did_token%2Btoken%26nonce%3DBYpSQXhp0np6Xa1H2CEBuyRkrV7Vrx9dC6nIzvEQ%26client_name%3DCasOAuthClient

- username Dagoret

- Surveys

https://www.eso.org/qi/


In [2]:
import matplotlib.pyplot as plt
from astropy.wcs import WCS

from astroquery.simbad import Simbad

from astroquery.eso import Eso
 
import astropy.coordinates as coordinates
from astropy.coordinates import Angle
import astropy.units as u

import h5py
import pandas as pd
import numpy as np


In [3]:
from astropy.table.pprint import conf
#conf.max_lines = -1
#conf.max_width = -1

In [4]:
def GetColumnHfData(hff,list_of_keys,nameval):
    """
    Extract hff atttribute 
    
    parameters
      hff           : descriptor of h5 file
      list_of_keys : list of exposures
      nameval      : name of the attribute
      
    return
           the array of values in the order of 
    """
    

    all_data = []
    for key in list_of_keys:
        group=hff.get(key)
        val=group.attrs[nameval]
        all_data.append(val)
    return all_data


## Fors2 catalog

In [5]:
input_file_h5  = '../data/FORS2spectra.hdf5'

In [6]:
hf =  h5py.File(input_file_h5, 'r') 
list_of_keys = list(hf.keys())

In [7]:
# pick one key    
key_sel =  list_of_keys[0]
# pick one group
group = hf.get(key_sel)  

In [8]:
#pickup all attribute names
all_subgroup_keys = []
for k in group.attrs.keys():
    all_subgroup_keys.append(k)

In [9]:
# create info
df_info = pd.DataFrame()
for key in all_subgroup_keys:
    arr=GetColumnHfData(hf, list_of_keys ,key)
    df_info[key] = arr

In [10]:
df_info.sort_values(by="num", ascending=True,inplace=True)

In [11]:
df_info_num = df_info["num"].values
key_tags = [ f"SPEC{num}" for num in df_info_num ]

In [12]:
df_info = df_info[['num', 'ra', 'dec', 'Rmag','redshift']]

In [13]:
df_info

,num,ra,dec,Rmag,redshift
51,2,13.52997,-28.44776,21.18,0.34555
112,3,13.52552,-28.42080,21.45,0.68660
545,9,13.50219,-28.40641,21.60,0.29266
17,13,13.49261,-28.38272,23.14,0.68870
43,19,13.60758,-28.43452,21.70,0.48395
...,...,...,...,...,...
531,734,13.68083,-28.23290,0.00,0.23012
532,735,13.69958,-28.23161,19.28,0.36169
533,736,13.66431,-28.22849,21.64,0.55777
534,737,13.62722,-28.22692,21.40,0.52001


## ESO catalog

In [20]:
#Eso.query_main()

In [17]:
print(Eso.list_surveys())

['081.C-0827', '092.A-0472', '096.B-0054', '1100.A-0528', '1101.A-0127', '193.D-0232', '195.B-0283', '196.B-0578', '196.D-0214', '197.A-0384', '198.A-0708', '60.A-9284H', '60.A-9493', 'ADHOC', 'ALCOHOLS', 'ALLSMOG', 'AMAZE', 'AMBRE', 'APEX-SciOps', 'ATLASGAL', 'ESPRESSO', 'ESSENCE', 'FDS', 'FEROS', 'FORS2-SPEC', 'Fornax3D', 'GAIAESO', 'GCAV', 'GIRAFFE', 'GOODS_FORS2', 'GOODS_ISAAC', 'GOODS_VIMOS_IMAG', 'GOODS_VIMOS_SPEC', 'GW170817', 'HARPS', 'HAWKI', 'HUGS', 'INSPIRE', 'KIDS', 'KMOS', 'LEGA-C', 'LESS', 'MUSE', 'MUSE-DEEP', 'MUSE-STD', 'MW-BULGE-PSFPHOT', 'NGTS', 'PESSTO', 'PHANGS', 'PIONIER', 'SHARKS', 'SPHERE', 'SUPER', 'UVES', 'UVES_SQUAD', 'UltraVISTA', 'VANDELS', 'VEGAS', 'VEILS', 'VEXAS', 'VHS', 'VIDEO', 'VIKING', 'VIMOS', 'VINROUGE', 'VIPERS', 'VISIONS', 'VMC', 'VPHASplus', 'VST-ATLAS', 'VVV', 'VVVX', 'XQ-100', 'XSGRB', 'XSHOOTER', 'XSL', 'ZCOSMOS']


In [18]:
print(Eso.list_instruments())

['fors1', 'fors2', 'sphere', 'vimos', 'omegacam', 'eris', 'hawki', 'isaac', 'naco', 'visir', 'vircam', 'apex', 'giraffe', 'uves', 'xshooter', 'espresso', 'muse', 'crires', 'kmos', 'sinfoni', 'amber', 'gravity', 'matisse', 'midi', 'pionier', 'wlgsu']


In [23]:
#Eso.query_instrument(instrument="fors2")

## Target

In [24]:
target_name_simbad = "BAX 013.5117-28.3994"
target_name_ned = "RX J0054.0-2823"

In [25]:
result_table = Simbad.query_object(target_name_simbad)

In [26]:
ra_str = result_table['RA'][0] + ' hours'
dec_str = result_table['DEC'][0] + ' degree'

In [27]:
coord = coordinates.SkyCoord(ra_str,dec_str)
coord

<SkyCoord (ICRS): (ra, dec) in deg
    (13.51166667, -28.39944444)>